In [69]:
#필요 패키지 import
import pandas as pd
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import time
import requests
import urllib3
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
ssl._create_default_https_context = ssl._create_unverified_context
headers = {"User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}

In [70]:
voter = pd.read_excel('voters.xlsx')

In [71]:
voter = voter[306:309]
voter

,이름,소속,직위,임시직위,이메일,핸드폰
306,박종화,공주대학교,조교수,교수,jonghwapark0@gmail.com,010-4328-2824
307,박주석,경희대학교,교수,교수,jspark@khu.ac.kr,010-5218-2672
308,박준철,강릉원주대학교,교수,교수,jcking@gwnu.ac.kr,010-2038-2377


In [72]:
#전체 교수 정보 setting
prof_total = []
for i in range(len(voter)):
    prof_total.append([voter.iloc[i, 0], voter.iloc[i, 1]])
prof_total

[['박종화', '공주대학교'], ['박주석', '경희대학교 '], ['박준철', '강릉원주대학교 ']]

네이버 프로필 크롤링

In [6]:
later_naver = [] #프로필이 없거나 오류가 난 행들 저장
#네이버 프로필 크롤링 위한 함수
def naver_crawl():
    try:
        url_naver = f'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query={voter.iloc[i, 1]}+{voter.iloc[i, 0]}'
        res = requests.get(url_naver, headers=headers, verify=False).content
        prof_soup = BeautifulSoup(res, "html.parser")
    except:
        time.sleep(3)
        url_naver = f'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query={voter.iloc[i, 1]}+{voter.iloc[i, 0]}'
        res = requests.get(url_naver, headers=headers, verify=False).content
        prof_soup = BeautifulSoup(res, "html.parser")    

    n = 0
    prof_naver = []
    while len(prof_soup.find_all('div', 'info_group')[n].find_all('dt')) > 0:
        n = n+1
    for j in range(n):
        prof_naver.append(prof_soup.find_all('div', 'info_group')[j].text)
    prof_total[i].append(prof_naver)
    if n == 0:
        later_naver.append(voter.iloc[i, 1]+" "+voter.iloc[i, 0])
        prof_total[i].remove([])

네이버 크롤링 완료함!!

돌리지 말 것

In [7]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
#네이버 크롤링 실행
for i in range(len(voter)):
    try:
        naver_crawl()
    except:
        later_naver.append(voter.iloc[i, 1]+" "+voter.iloc[i, 0])
        continue
prof_total

[['박아름', '용인예술과학대학교'],
 ['박영기', 'George Washington University'],
 ['박윤주',
  '서울과학기술대학교',
  [' 분야  사회과학 > 심리학  ',
   ' 소속서울과학기술대학교 ',
   ' 작품  학술정보 12건  ',
   ' 대표논문   Cost-sensitive case-based reasoning using a genetic algorithm: Application to medical diagnosis   ']],
 ['박재홍', '영진전문대학교 '],
 ['박정선', '명지대학교 '],
 ['박정훈', '서울대학교'],
 ['박종화', '공주대학교'],
 ['박주석',
  '경희대학교 ',
  [' 소속경희대학교(교수) ',
   ' 학력 캘리포니아대학교 버클리캠퍼스 대학원 경영학 박사  ',
   ' 경력 마이데이터 코리아 허브 대표 한국빅데이터학회 회장 한국EA학회 회장 한국경영학회 부회장 ']],
 ['박준철', '강릉원주대학교 '],
 ['박지용', 'University Of North Carolina at Greensboro'],
 ['박지혜', '고려대학교 경영대학'],
 ['박진수', '용인대학교'],
 ['박진수',
  '서울대학교 ',
  [' 소속서울대학교(교수), 한국증권선물거래소(위원) ',
   ' 학력 애리조나대학교 대학원 경영학 박사  ',
   ' 수상 2006년 한국경영정보학회 추계학술대회 우수 논문상 2008년 톰슨 로이터스 최우수 논문 2007년 한국지능정보시스템학회 춘계학술대회 우수 논문상 ',
   ' 경력 2006.05~2006.11 재정경제부 시장효율화위원회 IT 전문가소위원회 위원 2006.07~2006.12 한국증권예탁결제원 자문교수 ']],
 ['박진우',
  '서울대학교 ',
  [' 출생1952. ',
   ' 소속서울대학교(교수) ',
   ' 학력 캘리포니아대학교 버클리캠퍼스 대학원 산업공학 박사  ',
   ' 경력 2014 서울대학교 산업

In [8]:
%store prof_total

Stored 'prof_total' (list)


구글 크롤링

검색 후 맨 위에 뜨는 페이지를 기준으로

In [10]:
#selenium 위한 브라우저 열기
driver = webdriver.Chrome()
driver.implicitly_wait(2)
google_url = 'https://www.google.com/search?q=%27&sca_esv=564954462&hl=ko&source=hp&ei=c20BZf30OsCq0PEPjc-F0A0&iflsig=AO6bgOgAAAAAZQF7hIoHDk3NPXr59PTPTnC8ZegeGBsG&ved=0ahUKEwi9-vrCkKeBAxVAFTQIHY1nAdoQ4dUDCAs&uact=5&oq=%27&gs_lp=Egdnd3Mtd2l6IgEnSIErUMYbWLAocAR4AJABAJgBhgGgAYYBqgEDMC4xuAEDyAEA-AEBqAIA&sclient=gws-wiz&bshm=rimc/1'

In [11]:
#search를 위한 함수
def google_search():
    driver.get(google_url)
    driver.find_element(By.ID, 'APjFqb').clear()

In [12]:
#교수명으로부터 앞뒤로 텍스트 가져오되, 밑으로 행이 없을 경우 더 적은 행 가져오도록
def finder(x):
    #x에 들어가는 값: 해당 Html에서 교수명이 있는 index
    n = 0
    k = 0
    global list_
    list_ = []
    try:
        while n < 23:
            k += 1
            if len(page_text[x+k-1]) > 0:
                list_.append(page_text[x+k-2])
                n += 1
    except IndexError:
        while n < 4:
            k += 1
            if len(page_text[x+k-1]) > 0:
                list_.append(page_text[x+k-2])
                n += 1

In [13]:
prof_google = [] #구글 크롤링 완료된 목록
later_google = [] #구글 크롤링 안 되거나 오류난 목록

for loop in range(len(voter)):
    google_search()
    #검색 후 맨 위 검색결과 클릭
    if '교수' in [voter.iloc[loop, 3]]:
        driver.find_element(By.ID, 'APjFqb').send_keys(voter.iloc[loop, 1] + " " + voter.iloc[loop, 0]+" "+'교수')
    else:
        driver.find_element(By.ID, 'APjFqb').send_keys(voter.iloc[loop, 1] + " " + voter.iloc[loop, 0])
    driver.find_element(By.CLASS_NAME, 'Tg7LZd').click()
    driver.find_element(By.CLASS_NAME, 'MjjYud').find_element(By.TAG_NAME, 'span').click()

    try:
        search_url = driver.current_url

        #들어간 사이트 크롤링 시도
        prof_soup = BeautifulSoup(driver.page_source, 'html.parser')
        page_text = re.split(r'\n|\t|\r', prof_soup.text.replace(u'\xa0', u''))
        for i in range(page_text.count('')):
            page_text.remove('')

        #사이트명 추가
        title = prof_soup.find('title').text
        prof_total[loop].insert(2, title)

        #교수진 이름으로부터 아래 행들 크롤링
        name_dup = []
        for j in range(len(page_text)):
            if voter.iloc[loop, 0] in page_text[j]:
                name_dup.append(j)
                index1 = name_dup[0]
        if len(name_dup) > 1:
            index2 = name_dup[1]

        #홈페이지에 교수명 없을 경우
        if len(name_dup) == 0:
            prof_google.append([page_text])

        #교수명 1번 나올 경우    
        elif len(name_dup) == 1:
            finder(index1)
            prof_google.append(list_)

        #교수명 2번 나올 경우
        else:
            list__ = []
            finder(index1)
            list__ = [x for x in list_]
            finder(index2)
        #list__에 추가된 것이 교수 정보가 아닐 경우(교수명이 홈페이지 제목에 있을 경우 등)
            if '학사일정' in list_ or '시설 안내' in list_ or '인사말' in list_ or '행정실' in list_ or '공지사항' in list_ or 'Menu' in list_:
                list_ = []
            if '학사일정' in list__ or '시설 안내' in list__ or '인사말' in list__ or '행정실' in list__ or '공지사항' in list__ or 'Menu' in list_:
                list__ = []
                list__ = [x for x in list_]
            else:
                for n in range(len(list_)):
                    list__.append(list_[n])
            prof_google.append(list__)
    
    except IndexError:
        prof_google.append([page_text])
            
    except Exception as ex:
        prof_google.append('X')
        later_google.append([voter.iloc[loop, 1]+voter.iloc[loop, 0], f'{ex}'])
        print(f'{voter.iloc[loop, 1]+" "+voter.iloc[loop, 0]}', ex)


경희대학교  서영호 Alert Text: 글이 존재하지 않습니다.

글이 삭제되었거나 이동된 경우입니다.
Message: unexpected alert open: {Alert text : 글이 존재하지 않습니다.

글이 삭제되었거나 이동된 경우입니다.}
  (Session info: chrome=117.0.5938.150)
Stacktrace:
	GetHandleVerifier [0x00007FF610977D12+55474]
	(No symbol) [0x00007FF6108E77C2]
	(No symbol) [0x00007FF61079E0EB]
	(No symbol) [0x00007FF610818132]
	(No symbol) [0x00007FF6107FEF33]
	(No symbol) [0x00007FF6107D3D41]
	(No symbol) [0x00007FF6107D4F84]
	GetHandleVerifier [0x00007FF610CDB762+3609346]
	GetHandleVerifier [0x00007FF610D31A80+3962400]
	GetHandleVerifier [0x00007FF610D29F0F+3930799]
	GetHandleVerifier [0x00007FF610A13CA6+694342]
	(No symbol) [0x00007FF6108F2218]
	(No symbol) [0x00007FF6108EE484]
	(No symbol) [0x00007FF6108EE5B2]
	(No symbol) [0x00007FF6108DEE13]
	BaseThreadInitThunk [0x00007FF95AD1257D+29]
	RtlUserThreadStart [0x00007FF95B8CAA78+40]

단국대학교 서형호 'NoneType' object has no attribute 'text'
홍익대학교 신동익 'NoneType' object has no attribute 'text'
금오공과대학교  신호균 'NoneType' object has 

In [15]:
%store prof_google

Stored 'prof_google' (list)


In [16]:
driver.quit()

In [ ]:
prof_google

[[['김태형 | Facebook',
   "Facebook로그인로그인계정을 잊으셨나요?김태형친구사진동영상더 보기친구사진동영상김태형소개직장현 단국대학교 경영대학원 초빙교수 초빙교수2020년 2월 27일 - 현재단국대학교 경영대학원 초빙교수2020년 2월 26일 - 현재전 평화은행 근무에서 근무2019년 10월 8일 - 현재전 한국산업은행 근무에서 근무2019년 10월 8일 - 현재현 한국프로젝트출판사 전무이사 전무이사/경영학박사/단국대 경영대학원 교원2019년 6월 4일 - 현재전 우리금융그룹 우리FIS 검사부장서울대학한양대 MBA에서 MBA 전공단국대학교 대학원 박사과정에서 MIS 전공한양대학교 대학원에서 MBA 전공고등학교표시할 학교 정보 없음사진사진 모두 보기이름이 김태형인 다른 사람들김태형김태형김태형 부인 김태형김태형김태형Bãby Tîgër더 보기비슷한 이름을 가진 다른 사람들Gim Dela TorreGim TiarrAN IS ❤️🤡🌼😈হোঁ'পঁঁ খাঁ'কঁ'কিঁ মাঁ'গিঁGen GimA. Gim AsGim Mendez김태형개인정보처리방침 · 약관 · 광고 · AdChoices  · 쿠키 · 더 보기 · Meta © 2023김태형님과 Facebook에서 연결해보세요김태형님과 Facebook에서 연결해보세요이메일 또는 전화번호비밀번호로그인비밀번호를 잊으셨나요?또는새 계정 만들기",
   ' ']],
 ['나노양자전자 연구실',
  '김태환 명예교수',
  '02-2220-0354 twk@hanyang.ac.kr ',
  'http://quanta.hanyang.ac.kr ',
  '담당과목',
  '현대물리학, 고체전자물리개론, 반도체소자, 광전자공학 ',
  '학력',
  '뉴욕 주립대학교 물리학과 이학박사',
  '경력',
  '            2007 ~ 현재 한양대학교 HYU 석학교수 ',
  '2006 ~ 현재 한국과학기술 한림원 정회원 ',
  '2004 ~ 현재 한양대학교 공학대학원 전자공학 전공주임교수 ',
  '2003 ~ 

In [17]:
len(prof_google)

518

In [18]:
#구글 크롤링 결과 추가
for i in range(len(prof_total)):
    prof_total[i].insert(len(prof_total[i]), prof_google[i])
prof_total
#위 리스트 형식: [[이름, 소속, 사이트명(세부소속), [(있을 경우) 네이버 크롤링 결과], [구글 크롤링 결과]]]
#크롤링 결과 없을 경우 X 추가

[['박아름',
  '용인예술과학대학교',
  '빅데이터경영과(3년제)>학과소개>교수소개 ',
  ['교수소개',
   '박아름 학과장',
   '담당과목빅데이터 통계 기초, 빅데이터 분석, 디지털 마케팅, 경영통계, 스마트 비즈니스, 데이터 시각화',
   '연구실번호학술정보관 509호',
   '전화번호031-330-9245',
   '학력- 경희대학교 경영학과/주거환경학과 학사',
   '- 경희대학교 경영학과 석사',
   '- 경희대학교 경영학과 박사',
   '경력- 경희대학교 빅데이터 연구센터 연구교수',
   '- ㈜벤플 사물인터넷 서비스 기획',
   '- 한국표준협회 데이터 사이언스, 경영간부 강의',
   '- 경희대학교: 경영프로그래밍, MIS, 비즈니스 모델론, 디지털 미디어 경영, 창업과 비즈니스 실습',
   '- 단국대학교: 대학기초 SW입문, 창의적 사고와 코딩, AI활용',
   '- 경기대학교: 창의적 문제해결',
   '- 경희사이버대학교: 디지털 마케팅, 스마트 비즈니스',
   '- 고려사이버대학교: MIS, 신산업기술경영세미나 ',
   '- 서울디지털대학교: 4차산업혁명의 이해',
   '내용보기',
   '학력',
   '- 경희대학교 경영학과/주거환경학과 학사',
   '- 경희대학교 경영학과 석사',
   '- 경희대학교 경영학과 박사',
   '경력']],
 ['박영기',
  'George Washington University',
  '한국경영정보학회',
  [' ',
   '[부고] 박영기 교수님 (George Washington University) 부친상 訃告2023.10.01',
   '부고안내',
   ' ',
   ' 박영기 교수님 (George Washington University)',
   '부친상 訃告 ',
   ' ',
   '우리 학회의 종신회원이신 박영기 교수님(George Washington Univ.)께서 부친상을 ',
   '당하셨음을 알려드립니다.',
   ' ▶ 빈  소 : 양

In [19]:
%store prof_google

Stored 'prof_google' (list)


뉴스기사 크롤링

In [73]:
news_list = []
for i in range(len(voter)):
    news_list.append([voter.iloc[i, 0]])

In [74]:
#검색어 완전 일치 적용 (ex. 경희대 "이경전 교수")
#최근 5년 한정할 경우: 링크 뒤에 '&de=2023.10.12&ds=2016.10.12&pd=3' 붙이기
#최신순: 링크 뒤에 '&sort=1' 붙이기
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
for n in range(len(voter)):
    try:
        if '학교' in voter.iloc[n, 1] and voter.iloc[n, 3] == '교수':
            url_news = f'https://search.naver.com/search.naver?where=news&query={voter.iloc[n, 1].replace("학교", "")}+"{voter.iloc[n, 0]}+교수"'
        elif voter.iloc[n, 2] == '교수':
            url_news = f'https://search.naver.com/search.naver?where=news&query={voter.iloc[n, 1]}+"{voter.iloc[n, 0]}+교수"'
        else:
            url_news = f'https://search.naver.com/search.naver?where=news&query="{voter.iloc[n, 1]}+{voter.iloc[n, 0]}"'
        res = requests.get(url_news, headers=headers, verify=False).content
        news_soup = BeautifulSoup(res, "html.parser")

        nd = 1 # 뉴스 순서 (sp_nws)에 들어갈 숫자
        nt = 0 # nt가 2가 되면(기사 2개 가져오면) stop
        while nt < 2:
            try:
                news_date = news_soup.select_one(f'#sp_nws{nd} > div > div > div.news_info > div.info_group > span').text
                news_title = news_soup.select_one(f'#sp_nws{nd} > div > div > a').text
                news1 = news_soup.select_one(f'#sp_nws{nd} > div > div > a').attrs['href']
                res = requests.get(news1, headers=headers, verify=False).content
                news1_soup = BeautifulSoup(res, "html.parser")
                news1_txt = re.sub(r'\n|\t|\r', '', news1_soup.text)
                news_list[n].append([news_title])
                news_list[n][len(news_list[n])-1].insert(1, news_date)
                news_list[n][len(news_list[n])-1].insert(2, news1)
                news_list[n][len(news_list[n])-1].insert(3, news1_txt)
                nt += 1
                nd += 1
            except:
                nd += 1

        if '학교' in voter.iloc[n, 1] and voter.iloc[n, 3] == '교수':
            url_news = f'https://search.naver.com/search.naver?where=news&query={voter.iloc[n, 1].replace("학교", "")}+"{voter.iloc[n, 0]}+교수"&sort=1'
        elif voter.iloc[n, 2] == '교수':
            url_news = f'https://search.naver.com/search.naver?where=news&query={voter.iloc[n, 1]}+"{voter.iloc[n, 0]}+교수"&sort=1'
        else:
            url_news = f'https://search.naver.com/search.naver?where=news&query="{voter.iloc[n, 1]}+{voter.iloc[n, 0]}"&sort=1'
        res = requests.get(url_news, headers=headers, verify=False).content
        news_soup = BeautifulSoup(res, "html.parser")

        nd = 1 # 뉴스 순서 (sp_nws)에 들어갈 숫자
        nt = 0 # nt가 2가 되면(기사 2개 가져오면) stop
        while nt < 2:
            try:
                news_date = news_soup.select_one(f'#sp_nws{nd} > div > div > div.news_info > div.info_group > span').text
                news_title = news_soup.select_one(f'#sp_nws{nd} > div > div > a').text
                if news_title in news_list[n][1][0] or news_title in news_list[n][2][0]:
                    nd += 1
                    continue
                news1 = news_soup.select_one(f'#sp_nws{nd} > div > div > a').attrs['href']
                res = requests.get(news1, headers=headers, verify=False).content
                news1_soup = BeautifulSoup(res, "html.parser")
                news1_txt = re.sub(r'\n|\t|\r', '', news1_soup.text)
                news_list[n].append([news_title])
                news_list[n][len(news_list[n])-1].insert(1, news_date)
                news_list[n][len(news_list[n])-1].insert(2, news1)
                news_list[n][len(news_list[n])-1].insert(3, news1_txt)
                nt += 1
                nd += 1
            except:
                nd += 1
    except Exception as ex:
        print(f'{voter.iloc[n, 1]+" "+voter.iloc[n, 0]}', ex)

In [66]:
news_list

[['박종화',
  ['KBS 코로나19 뉴스, 누적 시청자 71억 명',
   '2022.01.20.',
   'http://star.mk.co.kr/new/view.php?mc=ST&year=2022&no=62646',
   'KBS 코로나19 뉴스, 누적 시청자 71억 명 - 스타투데이매경미디어그룹 바로가기뉴스증권부동산교육신문PDF구독신청더보기뉴스인기뉴스오피니언기획·연재스페셜에디션스타투데이스포츠게임진영문뉴스MK ChinaMK빌리어드뉴스미라클어헤드MK ESG 프리미엄매경e신문레이더M레이더P오늘의 매경뉴스레터들려주는 뉴스엠플러스  재테크증권부동산경제용어신설법인 라이프골프Book여행날씨우버人사이트MK생애설계센터M플레이Car 비즈니스The Biz Times닛케이텔레콤한국데이터거래소MK몰 서비스 전체보기본문 바로가기스타투데이 > KBS 코로나19 뉴스, 누적 시청자 71억 명기사입력 2022.01.20 21:45:34 재난방송주관방송사 KBS가 ‘코로나19 비상방송’을 통해 재난극복의 조력자로서 양적·질적 측면에서 충분히 기여했다는 평가를 담은 보고서가 나왔다. KBS 공영미디어연구소와 동덕여대 최믿음 교수, 공주대 박종화 교수가 공동 연구한 ‘코로나19 비상방송 성과분석’ 보고서는 2020년 1월 20일부터 2021년 12월 31일까지의 뉴스와 TV·라디오 프로그램 등을 대상으로 KBS가 코로나19 팬데믹 상황에서 얼마나 충실하게 재난 관련 정보를 제공하고 있는지, 시기별로 제공되는 뉴스와 정보의 질과 양은 적절했는지 등을 분석했다.보고서에 따르면 지난 2년간 KBS는 2만 4781건의 관련 기사를 3만 4978회 방송했다. TV 뉴스의 누적 시청자수는 71억 809만 명에 달하는 것으로 나타났다.KBS가 방송한 40개의 코로나19 TV 프로그램은 6억 8370만 명이 시청한 것으로 조사됐다. TV 뉴스와 프로그램의 누적 시청자 수를 합치면 77억 9179만 명이다. 이는 지난 2년간 우리 국민 1명이 KBS의 코로나19 방송을 평균 151회 이상 시청한 것이다

In [67]:
%store news_list

Stored 'news_list' (list)


교수 정보 크롤링 결과 & 뉴스기사 크롤링 결과 파일로 저장

In [24]:
df_prof = pd.DataFrame.from_records(prof_total)
df_prof.to_excel('(301~)교수정보크롤링.xlsx')

In [65]:
df_news = pd.DataFrame.from_records(news_list)
df_news.to_excel('(301~400)뉴스크롤링(다수).xlsx')

In [68]:
%store voter
%store prof_total
%store news_list

Stored 'voter' (DataFrame)


UsageError: Unknown variable 'prof_total'


In [29]:
%store

Stored variables and their in-db values:
backup_news                 -> [['김태형'], ['김태환', '한양大 김태환 교수 이달의 과학기술자상 수상', '201
gpt_news                    -> ['X', "한양대 김태환 교수는 나노 복합체를 이용해 차세대 비휘발성 메모리 소자를 제작
gpt_profile                 -> ['소속: 단국대학교, 한국프로젝트출판사\n세부소속: 경영대학원\n지역: 수도권\n학력: 
news_list                   -> [['박아름', '녹색금융·ESG 국제심포지엄 27일 코엑스서 개최', '2022.04.2
news_list_copy              -> [['Kwangjin Lee', 'https://www.hankyung.com/financ
prof_df                     ->      이름                소속              세부소속    직위 
prof_google                 -> [['교수소개', '박아름 학과장', '담당과목빅데이터 통계 기초, 빅데이터 분석, 디지털
prof_total                  -> [['박아름', '용인예술과학대학교', '빅데이터경영과(3년제)>학과소개>교수소개 ', [
prof_total_copy             -> [['Kwangjin Lee', 'Pepperdine University', [' ', '
voter                       ->       이름                                 소속       
